In [1]:
import numpy as np
import csv
import scipy.io
!pwd

/home/yousof/RideCellAssignment/scripts


In [2]:
csvFile = "res.txt"

In [3]:
# read the csv file
vectors = []
with open(csvFile) as f:
    csvReader = csv.reader(f, delimiter = ',')
    for row in csvReader:
        vectors.append([float(e) for e in row])
        
print(type (vectors[0][0]),vectors[0])

<class 'float'> [-0.0135694, 0.0484322, -0.998734, 0.985475, -0.121977, 0.118152, -1.69102]


In [4]:
vecs = np.array(vectors,dtype = np.float32)
print(vecs.shape)

(35, 7)


In [5]:
noramlFromCam = vecs[:,:3]
noramlFromPC = vecs[:,3:6]
print(noramlFromCam[0,:])
print(noramlFromPC[0,:])

[-0.0135694  0.0484322 -0.998734 ]
[ 0.985475 -0.121977  0.118152]


In [ ]:
# check the norm of the normal vectors
for i in range(vecs.shape[0]):
    print(i, 'from cam', np.linalg.norm(noramlFromCam[i,:]), 'from PC', np.linalg.norm(noramlFromPC[i,:]))

In [9]:
v_c = noramlFromCam[0,:]
v_l = noramlFromPC[0,:] * (-1)
# reform the equation according to the notebook for one example
A_c = np.array([[0, -v_c[2], v_c[1]],
               [v_c[2], 0 , -v_c[0]],
               [-v_c[1], v_c[0], 0]], dtype = np.float32)
A_l = np.zeros((3,9), dtype=np.float32)
expanded = np.expand_dims(v_l,0)
A_l[0,0:3] = expanded
A_l[1,3:6] = expanded
A_l[2,6:9] = expanded
print("A_c:")
print(A_c)
print("A_l:")
print(A_l)
A = np.matmul(A_c,A_l)
print ("A: ")
print(A)

A_c:
[[ 0.         0.998734   0.0484322]
 [-0.998734   0.         0.0135694]
 [-0.0484322 -0.0135694  0.       ]]
A_l:
[[-0.985475  0.121977 -0.118152  0.        0.        0.        0.
   0.        0.      ]
 [ 0.        0.        0.       -0.985475  0.121977 -0.118152  0.
   0.        0.      ]
 [ 0.        0.        0.        0.        0.        0.       -0.985475
   0.121977 -0.118152]]
A: 
[[ 0.          0.          0.         -0.9842274   0.12182258 -0.11800242
  -0.04772872  0.00590761 -0.00572236]
 [ 0.9842274  -0.12182258  0.11800242  0.          0.          0.
  -0.0133723   0.00165515 -0.00160325]
 [ 0.04772872 -0.00590761  0.00572236  0.0133723  -0.00165515  0.00160325
   0.          0.          0.        ]]


In [10]:
# make the big A matrix in equation AR = 0
A = np.empty((0,9), dtype=np.float32)
for idx in range(vecs.shape[0]):
    v_c = noramlFromCam[idx,:]
    v_l = noramlFromPC[idx,:]
    if (v_l[0]>0):
        v_l = v_l * (-1)
    v_c = v_c / np.linalg.norm(v_c)
    v_l = v_l /  np.linalg.norm(v_l)
#     print(v_c,v_l, np.linalg.norm(v_c), np.linalg.norm(v_l))
    A_c = np.array([[0, -v_c[2], v_c[1]],
                    [v_c[2], 0 , -v_c[0]],
                    [-v_c[1], v_c[0], 0]], dtype = np.float32)
    A_l = np.zeros((3,9), dtype=np.float32)
    expanded = np.expand_dims(v_l,0)
    A_l[0,0:3] = expanded
    A_l[1,3:6] = expanded
    A_l[2,6:9] = expanded
    a = np.matmul(A_c,A_l)
    A = np.concatenate((A,a),axis=0)
print (A.shape)
print (A[:6,:])
scipy.io.savemat('A.mat',mdict={'A': A}) # to check the svd in Matlab as well.

(105, 9)
[[ 0.          0.          0.         -0.984228    0.12182266 -0.1180025
  -0.04772875  0.00590762 -0.00572237]
 [ 0.984228   -0.12182266  0.1180025   0.          0.          0.
  -0.01337231  0.00165516 -0.00160325]
 [ 0.04772875 -0.00590762  0.00572237  0.01337231 -0.00165516  0.00160325
   0.          0.          0.        ]
 [ 0.          0.          0.         -0.98013526  0.11438649 -0.15066895
   0.03092683 -0.00360931  0.00475415]
 [ 0.98013526 -0.11438649  0.15066895  0.          0.          0.
  -0.0496547   0.00579494 -0.00763305]
 [-0.03092683  0.00360931 -0.00475415  0.0496547  -0.00579494  0.00763305
   0.          0.          0.        ]]


In [11]:
# calculate the SVD and solve for R in AR = 0
u,s,v = np.linalg.svd(A)
print("v:", v.shape)
print(v)
print("s:", s.shape)
print(s)
print("u:", u.shape)
print(u)

v: (9, 9)
[[-6.24727570e-02 -2.75775301e-03 -1.78898256e-02  9.42671537e-01
  -9.80177429e-03  2.67323285e-01  1.80390745e-01 -2.63732788e-03
   5.50845340e-02]
 [ 9.54319954e-01 -5.24748750e-02  2.67519057e-01  5.45265861e-02
  -2.53075603e-02  1.49208158e-02  4.47900556e-02  9.43625420e-02
   1.43762687e-02]
 [ 2.31708344e-02 -6.44713402e-01  1.70271173e-02  1.20051824e-01
   1.49038536e-02  2.84097567e-02 -6.11208379e-01 -3.89311969e-01
  -2.07211629e-01]
 [-8.31754296e-05  8.75722244e-02 -2.08794154e-05 -1.43764289e-02
  -9.73319769e-01  3.39309475e-03  1.34951128e-02 -2.11042821e-01
   6.35181135e-03]
 [-1.26455054e-01 -3.35427195e-01  8.23262557e-02  3.48264389e-02
  -2.24145964e-01 -1.02262087e-02 -2.02111557e-01  8.78671288e-01
  -1.57764722e-02]
 [ 1.81012601e-01  2.54117817e-01 -5.92207670e-01  2.35349134e-01
  -9.16665886e-03 -6.11060202e-01 -2.45146424e-01  9.92393568e-02
  -2.30511531e-01]
 [ 1.83967844e-01  2.01157853e-02 -6.70186400e-01 -1.75710753e-01
  -1.26555925e-02 

In [12]:
R_vec = v[-1,:]
print(R_vec)

[-0.02287787  0.05875027  0.08560541 -0.04251162 -0.00704137  0.19125067
  0.22945617  0.04902015 -0.94615537]


In [13]:
# reshape to R
R = R_vec.reshape((3,3))
print(R)

[[-0.02287787  0.05875027  0.08560541]
 [-0.04251162 -0.00704137  0.19125067]
 [ 0.22945617  0.04902015 -0.94615537]]


In [14]:
# Check how close it is to a rotation matrix now
def check(R):
    for i in range(3):
        n = np.linalg.norm(R[:,i])
        print("norm of column", i , "is" , n)

    print ("R*R.transpose= ")
    RRT = np.matmul(R,R.transpose())
    print (RRT)

check(R)

norm of column 0 is 0.23447978
norm of column 1 is 0.07683846
norm of column 2 is 0.9690795
R*R.transpose= 
[[ 0.01130328  0.01693098 -0.08336554]
 [ 0.01693098  0.03843364 -0.19105257]
 [-0.08336554 -0.19105257  0.9502631 ]]


In [15]:
# try to turn it to a rotational Matrix
U,S,V = np.linalg.svd(R)
R_new = np.matmul(U, V.transpose()) #.transpose()
print("S:",S)
print(R_new)

S: [0.99801016 0.06294069 0.00377406]
[[ 0.06455245  0.9959525  -0.06254325]
 [-0.9968653   0.06723084  0.04170914]
 [ 0.04574517  0.05965477  0.9971703 ]]


In [ ]:
check(R_new)

In [ ]:
print(np.matmul(A,v[-1,:]))

In [ ]:
import random

In [6]:
Rotations = []
losses = []
samples = []
n_samples = 4
for i in range(100):
    idxs = random.sample(range(vecs.shape[0]), n_samples)
    A = np.empty((0,9), dtype=np.float32)
    for idx in idxs:
        v_c = noramlFromCam[idx,:]
        v_l = noramlFromPC[idx,:]
        if (v_l[0]>0):
            v_l = v_l * (-1)
        v_c = v_c / np.linalg.norm(v_c)
        v_l = v_l /  np.linalg.norm(v_l)
        A_c = np.array([[0, -v_c[2], v_c[1]],
                        [v_c[2], 0 , -v_c[0]],
                        [-v_c[1], v_c[0], 0]], dtype = np.float32)
        A_l = np.zeros((3,9), dtype=np.float32)
        expanded = np.expand_dims(v_l,0)
        A_l[0,0:3] = expanded
        A_l[1,3:6] = expanded
        A_l[2,6:9] = expanded
        a = np.matmul(A_c,A_l)
        A = np.concatenate((A,a),axis=0)
    u,s,v = np.linalg.svd(A)
    R_v = v[-1,:]
    R = R_v.reshape((3,3))
    loss = np.linalg.norm(np.matmul(A,v[-1,:]))
    Rotations.append(R)
    losses.append(loss)
    samples.append(idxs)

NameError: name 'random' is not defined

In [ ]:
print(min(losses),"|",losses.index(min(losses)))

In [ ]:
print(losses)

In [ ]:
print(Rotations[87])
check(Rotations[87])

In [ ]:
U,S,V = np.linalg.svd(Rotations[0])
R_new = np.matmul(U, V)
print("S:",S)
print(R_new)

In [8]:
A = np.empty((3,0), dtype=np.float32)
B = np.empty((3,0), dtype=np.float32)
for idx in range(vecs.shape[0]):
    v_c = noramlFromCam[idx,:]
    v_l = noramlFromPC[idx,:]
    if (v_l[0]>0):
        v_l = v_l * (-1)
    v_c = v_c / np.linalg.norm(v_c)
    v_l = v_l /  np.linalg.norm(v_l)
    expanded_c = np.expand_dims(v_c,0)
    expanded_l = np.expand_dims(v_l,0)
    A = np.concatenate((A,expanded_c.reshape(3,1)),axis=1)
    B = np.concatenate((B,expanded_l.reshape(3,1)),axis=1)
    
u,s,vt = np.linalg.svd(np.matmul(B,A.transpose()))
mid = np.identity(3,dtype=np.float32)
mid[2,2] = np.linalg.det(np.matmul(u,vt))
print(mid)
R = np.matmul(np.matmul(vt.transpose(),mid),u.transpose())
print(R)

[[1.         0.         0.        ]
 [0.         1.         0.        ]
 [0.         0.         0.99999994]]
[[-0.08604091 -0.9962046   0.01316683]
 [ 0.07531615 -0.01968187 -0.99696535]
 [ 0.9934407  -0.08478814  0.07672376]]
